# Serializerによるモデルの保存

In [9]:
# coding: UTF-8

import chainer
from chainer import Variable, Chain, optimizers, serializers
import chainer.links as L
import chainer.functions as F

# Trainer用
from chainer.datasets import tuple_dataset
from chainer import training, iterators
from chainer.training import extensions

import numpy as np
from sklearn import datasets # Scikit Learn にIris dataset が含まれている

In [2]:
# Iris データの読み込み
iris_data = datasets.load_iris()

In [3]:
x = iris_data.data.astype(np.float32)
t = iris_data.target
n = t.size

In [4]:
# 教師データの下処理
t_matrix = np.zeros(3 * n).reshape(n, 3).astype(np.float32)
for i in range(n):
    t_matrix[i, t[i]] = 1.0

In [5]:
# 訓練用データとテスト用データ 半分が訓練用データで残りがテスト用データ
indexes = np.arange(n)
indexes_train = indexes[indexes%2 != 0]
indexes_test = indexes[indexes%2 == 0]

x_train = x[indexes_train, :] # 訓練用 入力
t_train = t_matrix[indexes_train, :] # 訓練用 正解
x_test = x[indexes_test, :] # テスト用 入力
t_test = t[indexes_test] # テスト用 正解

# tuple_dataset を使う
train = tuple_dataset.TupleDataset(x_train, t_train)
x_test_v = Variable(x_test)

In [6]:
# Chain の記述
class IrisChain(Chain):
    def __init__(self):
        super(IrisChain, self).__init__(
            l1 = L.Linear(4, 6),
            l2 = L.Linear(6, 6),
            l3 = L.Linear(6, 3),
        )
    
    def __call__(self, x, t):
        return F.mean_squared_error(self.predict(x), t)
        
    def predict(self, x):
        h1 = F.sigmoid(self.l1(x))
        h2 = F.sigmoid(self.l2(h1))
        h3 = self.l3(h2)
        return h3

In [7]:
# モデルとOptimizerの設定
model = IrisChain()
optimizer = optimizers.Adam()
optimizer.setup(model)

In [8]:
# 学習

# Trainer を使う
train_iter = iterators.SerialIterator(train, 30) # 1回の学習30個だけ使う
updater = training.StandardUpdater(train_iter, optimizer)
trainer = training.Trainer(updater, (5000, 'epoch'))
trainer.extend(extensions.ProgressBar())
trainer.run()

     total [..................................................]  0.80%
this epoch [..................................................]  0.00%
       100 iter, 40 epoch / 5000 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
     total [..................................................]  1.60%
this epoch [..................................................]  0.00%
       200 iter, 80 epoch / 5000 epochs
    342.47 iters/sec. Estimated time to finish: 0:00:35.916007.
     total [#.................................................]  2.40%
this epoch [..................................................]  0.00%
       300 iter, 120 epoch / 5000 epochs
    334.45 iters/sec. Estimated time to finish: 0:00:36.478018.
     total [#.................................................]  3.20%
this epoch [..................................................]  0.00%
       400 iter, 160 epoch / 5000 epochs
     333.7 iters/sec. Estimated time to finish: 0:00:36.259673.
     total [##.......

     total [##############....................................] 28.00%
this epoch [..................................................]  0.00%
      3500 iter, 1400 epoch / 5000 epochs
    262.14 iters/sec. Estimated time to finish: 0:00:34.332354.
     total [##############....................................] 28.80%
this epoch [..................................................]  0.00%
      3600 iter, 1440 epoch / 5000 epochs
    259.49 iters/sec. Estimated time to finish: 0:00:34.298058.
     total [##############....................................] 29.60%
this epoch [..................................................]  0.00%
      3700 iter, 1480 epoch / 5000 epochs
    254.45 iters/sec. Estimated time to finish: 0:00:34.584000.
     total [###############...................................] 30.40%
this epoch [..................................................]  0.00%
      3800 iter, 1520 epoch / 5000 epochs
    250.95 iters/sec. Estimated time to finish: 0:00:34.668325.
     tot

     total [###########################.......................] 55.20%
this epoch [..................................................]  0.00%
      6900 iter, 2760 epoch / 5000 epochs
    253.56 iters/sec. Estimated time to finish: 0:00:22.085412.
     total [############################......................] 56.00%
this epoch [..................................................]  0.00%
      7000 iter, 2800 epoch / 5000 epochs
    253.47 iters/sec. Estimated time to finish: 0:00:21.698696.
     total [############################......................] 56.80%
this epoch [..................................................]  0.00%
      7100 iter, 2840 epoch / 5000 epochs
    252.68 iters/sec. Estimated time to finish: 0:00:21.370886.
     total [############################......................] 57.60%
this epoch [..................................................]  0.00%
      7200 iter, 2880 epoch / 5000 epochs
    252.92 iters/sec. Estimated time to finish: 0:00:20.955155.
     tot

     total [#########################################.........] 82.40%
this epoch [..................................................]  0.00%
     10300 iter, 4120 epoch / 5000 epochs
    269.25 iters/sec. Estimated time to finish: 0:00:08.170800.
     total [#########################################.........] 83.20%
this epoch [..................................................]  0.00%
     10400 iter, 4160 epoch / 5000 epochs
    269.54 iters/sec. Estimated time to finish: 0:00:07.791000.
     total [##########################################........] 84.00%
this epoch [..................................................]  0.00%
     10500 iter, 4200 epoch / 5000 epochs
    269.93 iters/sec. Estimated time to finish: 0:00:07.409200.
     total [##########################################........] 84.80%
this epoch [..................................................]  0.00%
     10600 iter, 4240 epoch / 5000 epochs
    272.15 iters/sec. Estimated time to finish: 0:00:06.981550.
     tot

In [10]:
# モデルの保存
serializers.save_npz("my_iris.npz", model)